In [2]:
from pyspark import SparkConf, SparkContext

conf = SparkConf().setMaster("local").setAppName("restaurant-review-average")
sc = SparkContext(conf=conf)
sc

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/15 10:44:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


<SparkContext master=local appName=restaurant-review-average>

In [3]:
filepath = "/home/ubuntu/working/spark-examples/data/restaurant_reviews.csv"

In [4]:
lines = sc.textFile(f"file:///{filepath}")
lines.collect()

['id,item,cateogry,reviews,',
 '0,짜장면,중식,125,',
 '1,짬뽕,중식,235,',
 '2,김밥,분식,32,',
 '3,떡볶이,분식,534,',
 '4,라멘,일식,223,',
 '5,돈가스,일식,52,',
 '6,우동,일식,12,',
 '7,쌀국수,아시안,312,',
 '8,햄버거,패스트푸드,12,',
 '9,치킨,패스트푸드,23']

In [5]:
# header 제거
header = lines.first()
header

'id,item,cateogry,reviews,'

In [6]:
rows = lines.filter(lambda row : row != header)
rows

PythonRDD[3] at RDD at PythonRDD.scala:53

In [7]:
rows.collect()

['0,짜장면,중식,125,',
 '1,짬뽕,중식,235,',
 '2,김밥,분식,32,',
 '3,떡볶이,분식,534,',
 '4,라멘,일식,223,',
 '5,돈가스,일식,52,',
 '6,우동,일식,12,',
 '7,쌀국수,아시안,312,',
 '8,햄버거,패스트푸드,12,',
 '9,치킨,패스트푸드,23']

In [8]:
# 각 행을 파싱하기위한 Task(함수)
def parse(row):
    
    fields = row.split(",")
    
    # 카테고리 가져오기
    category = fields[2]
    
    # 리뷰 개수 가져오기
    review_count = fields[3]
    review_count = int(review_count)
    
    return category, review_count

In [9]:
# 테스트
parse('0,짜장면,중식,125')

('중식', 125)

In [10]:
category_reviews = rows.map(parse)
category_reviews

PythonRDD[4] at RDD at PythonRDD.scala:53

In [11]:
category_reviews.collect()

[('중식', 125),
 ('중식', 235),
 ('분식', 32),
 ('분식', 534),
 ('일식', 223),
 ('일식', 52),
 ('일식', 12),
 ('아시안', 312),
 ('패스트푸드', 12),
 ('패스트푸드', 23)]

In [12]:
# Key는 변경하지 않고, value만 변경하기 -> mapValues
# - 각 카테고리 별 리뷰 개수와 종류 개수

category_reviews_count = category_reviews.mapValues(lambda review_count : (review_count, 1))
category_reviews_count.collect()

[('중식', (125, 1)),
 ('중식', (235, 1)),
 ('분식', (32, 1)),
 ('분식', (534, 1)),
 ('일식', (223, 1)),
 ('일식', (52, 1)),
 ('일식', (12, 1)),
 ('아시안', (312, 1)),
 ('패스트푸드', (12, 1)),
 ('패스트푸드', (23, 1))]

In [13]:
reduce = category_reviews_count.reduceByKey(lambda x, y : (x[0] + y[0], x[1] + y[1]))
reduce.collect()

[('중식', (360, 2)),
 ('분식', (566, 2)),
 ('일식', (287, 3)),
 ('아시안', (312, 1)),
 ('패스트푸드', (35, 2))]

In [15]:
average = reduce.mapValues(lambda x : x[0] / x[1])
average.collect()

[('중식', 180.0),
 ('분식', 283.0),
 ('일식', 95.66666666666667),
 ('아시안', 312.0),
 ('패스트푸드', 17.5)]

In [16]:
sc.stop()